<a href="https://colab.research.google.com/github/Anubhav2017/Dna-ml-ddp/blob/master/approach_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
bases={'A':np.array([0,0,0,1]), 'C':np.array([0,0,1,0]), 'G':np.array([0,1,0,0]), 'T':np.array([1,0,0,0]), 'N':np.array([0,0,0,0]), 'Y':np.array([0,0,0,0]), 'R':np.array([0,0,0,0]), 'W':np.array([0,0,0,0]), 'S':np.array([0,0,0,0]), 'K':np.array([0,0,0,0])}

def one_hot_encode(seq, unique_elems,maxlen):
    seq_encoded=[]
    i=0
    for el in seq:
        # print(el)
        if i<maxlen:
            seq_encoded.append(unique_elems[el])
            i+=1
        
    return np.array(seq_encoded)

def one_hot_encode_2(y,num_classes):
    y_encoded=[]
    for el in y:
        y_encoded.append(np.eye(num_classes)[el])
    return np.array(y_encoded)



In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!pip install biopython

In [10]:
from Bio import SeqIO
unique_elems=dict()
x_data=[]
y_data=[]
minlen=100000
for sequence in SeqIO.parse('dataset.fasta', "fasta"):
    unique_elems[sequence.description.split()[1]+sequence.description.split()[2]]=0
    if len(sequence.seq)<minlen:
        minlen=len(sequence.seq)

# print(minlen)
print(unique_elems.keys())
i=0
for el in unique_elems.keys():
    unique_elems[el]=i
    i+=1
# print(unique_elems)
for sequence in SeqIO.parse('dataset.fasta', "fasta"):
    x_data.append(one_hot_encode(sequence.seq,bases,minlen))
    y_data.append(unique_elems[sequence.description.split()[1]+sequence.description.split()[2]])

x_data=np.array(x_data)
y_data=one_hot_encode_2(y_data,len(unique_elems))


print(x_data.shape)
    

dict_keys(['Acinetobacterbaumannii', 'Acinetobactercalcoaceticus', 'Bacteroidesfragilis', 'Candidaalbicans', 'Candidaauris', 'Candidaglabrata', 'Candidaparapsilosis', 'Cosenzaeamyxofaciens', 'Cryptococcusneoformans', 'Enterobactercloacae', 'Enterococcusfaecalis', 'Enterococcusfaecium', 'Escherichiacoli', 'Haemophilusinfluenzae', 'Klebsiellaaerogenes', 'Klebsiellaoxytoca', 'Klebsiellapneumoniae', 'Listeriamonocytogenes', 'Neisseriameningitidis', 'Pichiakudriavzevii', 'Thermoproteusthermophilus', 'Thermoproteustenax', 'Proteusalimentorum', 'Proteuscolumbae', 'Proteuscibarius', 'Proteusterrae', 'Motiliproteussediminis', 'Shimwelliapseudoproteus', 'Obesumbacteriumproteus', 'Proteusvulgaris', 'Proteusmirabilis', 'Proteushauseri', 'Proteuspenneri', 'Pseudomonasaeruginosa', 'Salmonellaenterica', 'Salmonellabongori', 'Serratiamarcescens', 'Staphylococcusaureus'])
(118, 1110, 4)


In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D,Reshape, LSTM
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
model=Sequential()
model.add(LSTM(units=256, input_shape=(x_data.shape[1],x_data.shape[2])))
model.add(Reshape((-1,8)))
model.add(Conv1D(filters=128, kernel_size=12, activation='relu'))
model.add(Flatten())
model.add(Dense(units=len(unique_elems), activation='softmax'))
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='loss', patience=2, mode='min')
history = model.fit(x_data, y_data, epochs=1000, verbose=1, validation_data= None)